In [1]:
import numpy as np
import tensorflow as tf
tf.config.list_physical_devices("GPU")

import matplotlib.pyplot as plt


In [2]:
# train_ds=tf.keras.utils.image_dataset_from_directory(

#     directory = '.\\archive (7)\\train',
#     labels='inferred',
#     label_mode = 'int',
#     batch_size=32,
#     image_size=(150,150)
# )
# val_ds=tf.keras.utils.image_dataset_from_directory(

#     directory = '.\\archive (7)\\valid',
#     labels='inferred',
#     label_mode = 'int',
#     batch_size=32,
#     image_size=(150,150)
# )

# test_ds=tf.keras.utils.image_dataset_from_directory(

#     directory = '.\\archive (7)\\test',
#     labels='inferred',
#     label_mode = 'int',
#     batch_size=32,
#     image_size=(150,150)
# )


In [20]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    '.\\archive (7)\\train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

validation_gen = train_datagen.flow_from_directory(
    '.\\archive (7)\\valid',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

test_gen = test_datagen.flow_from_directory(
    '.\\archive (7)\\test',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels


Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


In [21]:
import os

directory = '.\\archive (7)\\valid'
num_folders = len([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))])
print("Number of folders in directory:", num_folders)

Number of folders in directory: 100


In [22]:
# # Define the input shape
# input_shape = (150, 150, 3)  # Assuming images are 150x150 pixels with 3 color channels (RGB)

# # Create the CNN model
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     # tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(num_folders, activation='softmax')  
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',  
#               metrics=['accuracy']
#               )

In [23]:
from keras.applications.vgg16 import VGG16

In [24]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [25]:
# for layer in conv_base.layers:
#     layer.trainable=False
conv_base.trainable=False

In [26]:
# conv_base.trainable = True

# set_trainable = False

# for layer in conv_base.layers:
#   if layer.name == 'block5_conv1':
#     set_trainable = True
#   if set_trainable:
#     layer.trainable = True
#   else:
#     layer.trainable = False

# for layer in conv_base.layers:
#   print(layer.name,layer.trainable)

In [31]:
model = tf.keras.Sequential()

model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(num_folders,activation='softmax'))

In [32]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy']
              )

In [33]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               2097408   
                                                                 
 dense_7 (Dense)             (None, 100)               25700     
                                                                 
Total params: 16,837,796
Trainable params: 2,123,108
Non-trainable params: 14,714,688
_________________________________________________________________


In [34]:
model.fit(train_gen,epochs=10,validation_data=validation_gen)

Epoch 1/10
422/422 [==============================] - 114s 256ms/step - loss: 2.9720 - accuracy: 0.3064 - val_loss: 1.8780 - val_accuracy: 0.4980
Epoch 2/10
422/422 [==============================] - 115s 272ms/step - loss: 1.6545 - accuracy: 0.5651 - val_loss: 1.4599 - val_accuracy: 0.5880
Epoch 3/10
422/422 [==============================] - 141s 333ms/step - loss: 1.2609 - accuracy: 0.6539 - val_loss: 1.3014 - val_accuracy: 0.6420
Epoch 4/10
422/422 [==============================] - 145s 344ms/step - loss: 1.0520 - accuracy: 0.7120 - val_loss: 1.1236 - val_accuracy: 0.6840
Epoch 5/10
422/422 [==============================] - 150s 355ms/step - loss: 0.8950 - accuracy: 0.7474 - val_loss: 1.1694 - val_accuracy: 0.6580
Epoch 6/10
422/422 [==============================] - 126s 299ms/step - loss: 0.7749 - accuracy: 0.7780 - val_loss: 1.1791 - val_accuracy: 0.6640
Epoch 7/10
422/422 [==============================] - 89s 210ms/step - loss: 0.6949 - accuracy: 0.8000 - val_loss: 1.0982 - 

In [35]:
model.evaluate(test_gen)

16/16 [==============================] - 3s 157ms/step - loss: 0.7215 - accuracy: 0.7780


[0.7215368747711182, 0.777999997138977]